In [1]:
import numpy as np

import time
import os
from os import listdir
from os.path import isfile, join

import glob
import string
from nltk.stem.porter import *

In [2]:
def directory_listing(dir_path):
    '''
    Input:  string (path to directory)
    Output: list of strings (full paths to files in the directory)
    '''
    
    return glob.glob(dir_path + '*.txt')

In [3]:
path = os.getcwd() +'/HillaryEmails/'
all_files = directory_listing(path)
#all_files

In [4]:
def read_file(file_path):
    '''
    Input:  string (full path to file)
    Output: string/text (full contents of a file)
    '''
    
    lines = [line.rstrip('\n') for line in open(file_path)]
    return ' '.join(lines)

In [5]:
file_text = read_file(all_files[0])
file_text

'UNCLASSIFIED U.S. Department of State Case No. F-2015-04841 Doc No. C05739714 Date: 05/13/2015 STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM. SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. RELEASE IN FULL From: Mills, Cheryl D <MillsCD©state.gova Sent Sunday, October 14, 2012 9:46 AM To: Subject: Re: Bloomberg: Libyan Ambassador\'s Death Not a Political Issue, Says Dad In car driving so can in a bit From: H [mallto:H0R22@clintonemail.corri] Sent: Sunday, October 14, 2012 09:39 AM To: Mills, Cheryl D Subject: Re: Bloomberg: Libyan Ambassador\'s Death Not a Political Issue, Says Dad Very nice. Can you talk? From: Mills, Cheryl D [mailto:MillsCD@state,gov] Sent: Sunday, October 14, 2012 09:00 AM To: H Subject Fw: Bloomberg: Libyan Ambassador\'s Death Not a Political Issue, Says Dad , From: Hensman, Chris D Sent: Sunday, October 14, 2012 08:19 AM To: Coordination Cc: Mills, Cheryl D; Reines, Philippe I Subject: Fw: Bloomberg: Libyan Ambassador\'s Death 

In [6]:
def tokenization(text, file_path):
    '''
    Input:  text(file contents), string (document id = path to file)
    Output: list of pairs < string(token) , string (document id) >
    '''
    doc_id = int(os.path.basename(file_path).replace('.txt','')) #retrieve document id from file path
    tokens = text.split()
    return [(token, doc_id) for token in tokens]
    

In [7]:
token_pairs = tokenization(file_text, all_files[0])
print("length of pairs: ", len(token_pairs))
#token_pairs

length of pairs:  1326


In [8]:
def linguistic_modules(token_pairs):
    '''
    Input:  list of pairs < token , document id >
    Output: list of pairs < modified token , document id >
    
    modified token: removing all punctuation symbols (!@#$%^&*()-_=+’`~”:;/.,?[]{}<>),lowercasingand stemming.
    '''
    
    stemmer = PorterStemmer()  
    return [(stemmer.stem(token.translate(str.maketrans('','',string.punctuation)).lower()), doc_id)  
            for token, doc_id in token_pairs  
                if token.translate(str.maketrans('','',string.punctuation)) is not ''] #if statement to check empty token
    

In [9]:
modified_token_pairs = linguistic_modules(token_pairs)
print('length after modification: ', len(modified_token_pairs))
#modified_token_pairs

length after modification:  1313


In [10]:
l = [('unclassifi', '133'),('us', '133'),('depart', '133'), ('!','12')]
linguistic_modules(l)

[('unclassifi', '133'), ('us', '133'), ('depart', '133')]

In [11]:
def sort_tokens(token_pairs):
    '''
    Input:  list of pairs < token , document id >
    Output: sorted list of pairs < token , document id >
    
    perform sorting of the token list: first by tokens (alphabetical order), and then by document ids 
    '''
    
    return sorted(token_pairs, key=lambda element: (element[0], element[1]))

In [12]:
sorted_modified_token_pairs = sort_tokens(modified_token_pairs)
#sorted_modified_token_pairs

In [13]:
def transformation_into_postings(sorted_token_pairs):
    '''
    Input: sorted list of pairs < token , document id >
    Output: inverted index
    
    Used dictionary data structure (Hash table)
    '''
    
    dictionary_ = {}
    for a, b in sorted_token_pairs:
        dictionary_.setdefault(a, []).append(b)
    return dictionary_

In [14]:
posting_list = transformation_into_postings(sorted_modified_token_pairs)
#posting_list

In [17]:
# MAIN
#start = time.time()

path = os.getcwd() +'/HillaryEmails/'
all_files = directory_listing(path)
#all_files = all_files[:10]

all_token_pairs = []

for i in range(len(all_files)):
    file_text = read_file(all_files[i])
    token_pairs = tokenization(file_text, all_files[i])
    modified_token_pairs = linguistic_modules(token_pairs)
    all_token_pairs = all_token_pairs + modified_token_pairs

#end = time.time()
#print(end - start)

In [18]:
#start = time.time()

sorted_token_pairs = sort_tokens(all_token_pairs)
posting_list = transformation_into_postings(sorted_token_pairs)
posting_list


#end = time.time()
#print(end - start)

{'002105': [4262],
 '05132015': [133, 133, 133, 133, 133, 133],
 '06302015': [483, 483, 1526, 1526, 1840, 1840, 1840, 1840],
 '07312015': [2123, 2123, 2566, 2566],
 '0758': [133],
 '08': [1840],
 '0819': [133],
 '08312015': [4262,
  4262,
  5619,
  5619,
  5619,
  5619,
  6811,
  6811,
  6811,
  6811,
  7604,
  7604],
 '09': [1840],
 '0900': [133],
 '0939': [133],
 '1015': [483],
 '1022': [483],
 '1030': [2566],
 '1049': [1526],
 '11': [133, 1526],
 '1100': [1840],
 '1103': [1526],
 '1109': [1526],
 '113': [4262],
 '12': [133, 483],
 '12110': [6811],
 '14': [133, 133, 133, 133, 133, 133],
 '154516': [1840],
 '16member': [133],
 '19': [5619],
 '193102': [7604],
 '194634': [6811],
 '20': [2566, 2566],
 '20037': [1840],
 '2005': [5619],
 '2009': [483, 1526, 1526, 1526, 1526, 1840, 1840, 1840, 2123, 2566, 2566],
 '2010': [4262, 4262, 5619, 5619, 6811, 6811, 6811, 7604, 7604],
 '2011': [133],
 '2012': [133, 133, 133, 133, 133, 133],
 '202': [1840, 1840],
 '21': [1526, 1526, 1526, 1526, 6811